In [116]:
import sys
sys.path.append('/home/woody/iwso/iwso092h/mad_ucb_kaggle')
import pandas as pd
import timm
import os
import torchvision.models as models
from tqdm import tqdm
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
# from datasets import CustomCancerDataset
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
mlb = MultiLabelBinarizer()
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from trainer import train_classification_model
import torch
import pandas as pd
import os
from torch.utils.data import Dataset
from PIL import Image  
from torchvision import transforms

root = r'/home/woody/iwso/iwso092h/mad_ucb_kaggle'
train_images = r'train_thumbnails'
image_folder = r'/home/woody/iwso/iwso092h/mad_ucb_kaggle/train_thumbnails'

class CustomCancerDataset(Dataset):
    def __init__(self, metadata_df, image_folder, transform=None):
        self.metadata_df = metadata_df
        self.image_folder = image_folder
        self.transform = transform  # Use the provided transform

    def __len__(self):
        return len(self.metadata_df)

    def __getitem__(self, idx):
        image_ids = self.metadata_df.image_id[idx]  
        image_name = os.path.join(self.image_folder, "{}_thumbnail.png".format(image_ids))
        image = Image.open(image_name).convert('RGB')
        label = int(self.metadata_df.label[idx])
        # label_CC  = self.metadata_df.label_CC[idx].astype(int)  
        # label_EC  = self.metadata_df.label_EC[idx].astype(int)    
        # label_HGSC  = self.metadata_df.label_HGSC[idx].astype(int)    
        # label_LGSC  = self.metadata_df.label_LGSC[idx].astype(int)    
        # label_MC  = self.metadata_df.label_MC[idx].astype(int)    

        if self.transform:
            image = self.transform(image)

        return image, label
        # return image, torch.tensor([label_CC, label_EC, label_HGSC, label_LGSC, label_MC], dtype=torch.float)
        
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        pt = torch.exp(-nn.CrossEntropyLoss(reduction='none')(inputs, targets))
        loss = (1 - pt) ** self.gamma * nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        if self.alpha is not None:
            loss = loss * self.alpha
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

train_df = pd.read_csv(os.path.join(root,'data','train.csv'))
train_wo_tma = train_df[train_df.is_tma==False]
label_mapping = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
train_wo_tma['label'] = train_wo_tma['label'].map(label_mapping)
train_split, val_split = train_test_split(train_wo_tma, test_size=0.2, random_state=17, stratify=train_wo_tma.label)
train_split = train_split.reset_index(drop=True)
val_split = val_split.reset_index(drop=True)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),# Color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48828688, 0.42932517, 0.49162089], std=[0.41380908, 0.37492874, 0.41795654])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48828688, 0.42932517, 0.49162089], std=[0.41380908, 0.37492874, 0.41795654])
])

train_ucb_dataset = CustomCancerDataset(train_split, image_folder='/home/woody/iwso/iwso092h/mad_ucb_kaggle/train_thumbnails',
                                  transform=train_transforms)
val_ucb_dataset = CustomCancerDataset(val_split, image_folder='/home/woody/iwso/iwso092h/mad_ucb_kaggle/train_thumbnails',
                                  transform=val_transforms)

# class_weights = {0: 0.46067416, 1: 2.48484848, 2: 0.82828283, 3: 1.15492958, 4: 2.82758621}
# sample_weights = [class_weights[label] for _, label in train_ucb_dataset]

# sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

batch_size = 16
train_loader = DataLoader(train_ucb_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ucb_dataset, batch_size=batch_size)


/tmp/ipykernel_1720223/3035528147.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_wo_tma['label'] = train_wo_tma['label'].map(label_mapping)


In [118]:
num_classes = 5  
learning_rate = 0.01
step_size = 3
gamma = 0.1  
num_epochs = 10
########################################

# resnet_model = models.resnet34(pretrained=True)
# in_features = resnet_model.fc.in_features
# resnet_model.fc = nn.Linear(in_features, num_classes)
model = UBCModel('efficient_net',num_classes=5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

/home/woody/iwso/iwso092h/miniconda/lib/python3.11/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


In [119]:
model.train()
running_loss = 0.0
total_samples = 0
all_labels = []
all_predictions = []

for inputs, labels in tqdm(train_loader, leave=False):
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(torch.argmax(outputs, 1).cpu().numpy())

    total_samples += labels.size(0)

In [120]:
model.eval()
running_loss = 0.0
total_samples = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        running_loss += loss.item()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(torch.argmax(outputs, 1).cpu().numpy())

        total_samples += labels.size(0)


In [117]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'
                
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=False, checkpoint_path=None):
        super(UBCModel, self).__init__()
        self.model = timm.create_model('tf_efficientnetv2_s_in21ft1k', pretrained=pretrained)

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        return output